<img style="float: right;"  src="images/LogoP.jpg" width="200">

# SLab Demo : DAC Analysis

This a **demo** Jupyter Notebook for the SLab projects

Version 1.0 (29/5/2018) License information is at the end of the document

---

## Introduction

This document analyzes the operation of **hardware board** DACs

You can apply the measurements to any harware board, but the comments about the measurements are only guaranteed to be valid when a **STM32 Nucleo64 F303RE** board is used. The comments also assume that you are using an **uncalibrated** board.

The **SLab** system uses, at least two DACs. In circuit schematics they are shown as in the figure below:

![Image01](images\DAC_Analysis\Image01.png)

In the examples we use a yellow jumper wire for DAC1 and an orange wire for DAC2, so the symbols are colored in the same way.

We show and model the DACs as if they were ideal sources but they are not ideal. No real component can ever be ideal. This document explores the operation limits of the DACs at DC or low frequency operation.

From measurements performed on the **hardware board**, a model that describes the real DAC operation will be obtained.

## Import and Connect

As always, in order to use the **SLab** system, we need to import the **SLab** module and **connect** to the board.

As we will use the [numpy](http://www.numpy.org/) module for calculations, we will also import it.

In [ ]:
# Import Numpy
import numpy as np

# Import all slab modules
import slab

In [ ]:
boardFolder = ''                                # Board folder (leave '' if you use only one board)
slab.setFilePrefix('../Files/')                 # Set File Prefix
slab.setCalPrefix('Calibrations/'+boardFolder)  # Set Calibration Prefix         
slab.connect()                                  # Connect to the board

## Measure and Calculate Cells 

This is Jupyter Python Notebook, as that, it includes both **Text Cells** that are like this one, only to be read, and **Code Cells** that include Python code.  
We will divide the **Code Cells** in two kinds:

* Measure Cells
* Calculate Cells

In the **Measure Cells** we will perform measurements on the hardware board. That means that you will be required to set the proper circuit connections and perform the measurement by executing the code cell.

In the **Calculate Cells** we will perform calculations on measurements. That means that we don't interact with the board or the circuit on those cells.

Separating those two kinds of cells eases the change of the calculation without needing to perform new measurements.

## Interactive plots

Executing the following code we can make our plots interactive. That way you can zoom and pan on the plot.

When you are done with the interaction, just hit the close interaction button to reduce the computing load.

If you don't like the interaction, just don't execute this code.

In [ ]:
# Make plots interactive
slab.interactivePlots()
%matplotlib notebook

## Open Circuit

The first thing we will do is to show the open circuit operation of the DAC. To do that, we will just connect the **DAC1** output to the **ADC2** input.

![Image03](images\DAC_Analysis\Image02.png)

In all this document we only use one **ADC**. For historical reasons that don't relate to this particular demo we use **ADC2**, but we could use any other **ADC**. In the F303RE board it is best no to use **ADC4** or **ADC8** because they are not buffered. That means that they have higher equivalent input resistance.

From the measurements we will obtain the **open circuit** DAC1 voltage **Voc** for all programming voltages **Vset** from 0.1V to 3.3V. Then we will compare the measurements against the program voltage.

We don't include the 0.0V in the input range to prevent a divide by zero later when we process the measurement data.

In [ ]:
# MEASURE

# Perform a DC sweep measurement 
dataOC = slab.dcSweep(1,0.1,3.3,0.1)

In [ ]:
# CALCULATE

# Define Vset and Voc from the obtained data
Vset = dataOC[0]  # DAC Goal
Voc  = dataOC[2]  # ADC2

# Plot Vset and Voc against Vset
slab.plot1n(Vset,[Vset,Voc],'Open circuit measurement','Vset (V)','Vset and Voc (V)'
        ,['Vset','Voc'])

# Calculate and plot the voltage difference
Vdif = Vset - Voc
slab.plot11(Vset,Vdif,'Voltage difference','Vset (V)','Vset - Voc (V)')

As you can see there is constant difference between **Vset** and the **Voc** output of DAC1. In my particular case it was about 50mV. That means that the DAC output is always 50mV below the programmed value.

As in the **SLab** experiments we always try to measure the DAC outputs, this error is not really a problem in most cases.

## Loaded Measurement

Now we will add a load to the DAC1 output.

![Image03](images\DAC_Analysis\Image03.png)

We use a $330\Omega$ value because it suits the **F303** board. For other boards, other values could be better.

Then we repeat the measurement and show again the **Vr** loaded DAC1 voltage against the **Vset** programmed voltage.

Afte the DC sweep we use the **slab.zero** function to set the DACs to zero and prevent stressing the loade DAC outputs.

In [ ]:
# MEASURE

# Repeat the DC sweep with the loaded DAC1
dataR = slab.dcSweep(1,0.1,3.3,0.1)

# Set DACs to zero
slab.zero()

In [ ]:
# CALCULATE

# Value of the resistor we use
Rvalue = 330

# Obtain loaded output Vr as a variable
# We convert the list to a numpy array to ease future calculations
Vr = np.array(dataR[2])

# Plot Vset and Vr against Vset
slab.plot1n(Vset,[Vset,Vr],'Loaded DAC measurement','Vset (V)','Vset and Vr (V)'
        ,['Vset','Vr'])

You can see that, for voltages below 2V, there is a constant difference between the loaded DAC output and the programmed voltage.

The difference is in the same range of the unloaded **Voc** measurement. About 50mV in my case.

For voltages above 2V we can see that the DAC voltage don't go up any further and is kept constant regardless of the **Vset** voltage.

We see that DAC voltage is limited at high voltages. Perhaps there is a limit on the DAC current output?

## DAC Current Limit

In order to obatin a better insight on the DAC limits, we can obtain and plot the current provided by the DAC:

$\qquad I_{DAC} = \frac{V_r}{R_1}$


In [ ]:
# CALCULATE

# Current calculation
i = Vr/Rvalue

# Current plot
slab.plot11(Vset,1000*i,'Current calculation','Vset (V)','I (mA)')

We can see that the current goes up until about 6.5mA and then it becomes constant.

One hypothesis will be that DAC1 has a 6.5mA current output limit.

To verify the hypothesis, we will repeat the measurements changing the resistance for one with a $470\Omega$ value.

![Image04](images\DAC_Analysis\Image04.png)


In [ ]:
# MEASURE

# Repeat the DC sweep with the new resistor
dataR2 = slab.dcSweep(1,0.1,3.3,0.1)

# Set DACs to zero
slab.zero()

In [ ]:
# CALCULATE

# Value of the new resistor
Rvalue2 = 470

# Obtain the new Vr2 as a variable
Vr2 = np.array(dataR2[2])

# New current calculation
i2 = Vr2/Rvalue2

# Plot Vset, Vr and Vr2 against Vset
slab.plot1n(Vset,[Vset,Vr,Vr2],'Loaded DAC measurement','Vset (V)','Vset, Vr2 (V)'
        ,['OC','330R','470R'])

# Plot both currents
slab.plot1n(Vset,[1000*i,1000*i2],'Current comparison','Vset (V)','I (mA)',['330R','470R'])

We can see that the maximum current for the $330\Omega$ resistor is not the same than for the $470\Omega$ resistor.

That means that the saturation of the DAC output voltage cannot be modelled by a current limitation on the DAC output.

## Model Proposal

We know that the F303 microcontroller (MCU) includes **drivers** for the DAC outputs. We don't know what kind of **drivers** are they, but we know that this MCU includes several operational amplifiers  (opamps) because we use them to drive the ADCs.

An hypothesis is that an opamp follower circuit is used to drive the **buffered** output from the internal **unbuffered** DAC.

![Image05](images\DAC_Analysis\Image05.png)

The opamp can be limited by its maximum output current of by its output resistance. We know that we are not, in this case, current limited because we know that the two used resistors give different output current limits. That's why we consider the output resistance of the opamp $R_{OUT}$ as the main source of non ideality for the opamp.

As we know that, for small loads, there is a $50mV$ difference between the DAC programmed value and the ADC measured value, we model that as an **offset voltage** $V_{OFFS}$

Now it is time to obtain the parameters of this model. The offset voltage is easy, as we have obtained it from the unloaded measurement:

$\qquad V_{OFFS} = 50 mV$

The output resistance can be calculated from the saturated region of the curves. The maximum opamp voltage is Vdd ($3.3 V$ in the F303 board). So the current in the saturation region is:

$\qquad I_{MAX} = \frac{V_{DD}}{R_{OUT}+R_{EXT}}$

Where $R_{EXT}$ is the $330\Omega$ or $470\Omega$ external resitor we are using.

We can calculate the internal resistance from the above formmula:

$\qquad R_{OUT} = \frac{V_{DD}}{I_{MAX}} - R_{EXT}$

Let's calculate this resistor from the measurement data.

In [ ]:
# Vdd value
Vdd = 3.3

# For the 330R case
Rout = Vdd/i - Rvalue

# For the 470R case
Rout2 = Vdd/i2 - Rvalue2

# As we need only the saturated regions
# we will only use data from Vset over 2.5V 

Vs   =  [V for R,V in zip(Rout,Vset) if V > 2.5]
Rout =  [R for R,V in zip(Rout,Vset) if V > 2.5]
Rout2 = [R for R,V in zip(Rout2,Vset) if V > 2.5]

# Show both calculations
slab.plot1n(Vs,[Rout,Rout2],'Internal Opamp Resistance'
              ,'Vset (V)','Rout (Ohm)',['330R','470R'])

We don't get a constant result, but the $5\Omega$ variation we see is only a 3% of the mean $170\Omega$ value.  
So, we set this value for our model:

$\qquad R_{OUT} = 170 \Omega$

## Check of the model

We can now check the model against our measurement data.
In order to do that we can write a function that obtains the DAC output from the programmed value and the output resistance $R_{EXT}$:

The voltage at the non inverting input of the opamp will be:

$\qquad Vnii = V_{SET} - V_{OFFS}$

The current output, if the opamp behaves as a follower will be:

$\qquad I_{OUT} = \frac{Vnii}{R_{EXT}}$

The voltage at the opamp output takes into account the internal opamp resitance:

$\qquad V_{OA} = I_{OUT} \; (R_{OUT}+R_{EXT})$

But it is limited to $V_{DD}$.

If the above $V_{OA}$ is less than $V_{DD}$, we get $Vnii$ as the DAC output. Otherwise, the DAC output can be calculated:

$\qquad V_{DAC} = \frac{V_{DD} \cdot R_{EXT}}{R_{OUT}+R_{EXT}}$

We will first develop this model and then we will compare it to the real measurements

In [ ]:
# DAC model parameters
Vdd   = 3.3
Voffs = 0.05
Rout  = 170

# DAC model
# Gives the DAC voltage from Vset and Rext
def DACmodel(Vset,Rext):
    Vnii = Vset - Voffs
    Iout = Vnii/Rext
    Voa = Iout*(Rext+170)
    if Voa < Vdd:
        return Vnii 
    return Vdd*Rext/(Rout+Rext)

# Apply the model to the 330R case
VrM = [DACmodel(V,Rvalue) for V in Vset]

# Apply the model to the 470R case
Vr2M = [DACmodel(V,Rvalue2) for V in Vset]

# Plot Vr and Vr2 and the model results against Vset
slab.plot1n(Vset,[Vr,Vr2,VrM,Vr2M],'Loaded DAC measurement','Vset (V)','Vset, Vr2 (V)'
        ,['330R Meas','470R Meas','330R Model','470R Model'])

We can **zoom** in the upper region of the graph, in interactive mode, to see it better

From the graph it seems that the model agrees with the measurement data.  
We can obtain a better assess the match by calculating the the model error.

In [ ]:
Error  = Vr  - VrM   # 330R
Error2 = Vr2 - Vr2M  # 470R

# Plot the errors
slab.plot1n(Vset,[Error,Error2],'Model error','Vset (V)','Error (V)',['330R','470R'])

As we can see the error is below $50 mV$

## DAC2 Operation

We can assume that the second DAC is similar to the first one.

This DAC, available on the MCU pin **PA5**, is connected, as the Nucleo64 manual shows in its schematics, to the **user LED**.

![Schematic](images\DAC_Analysis\Schematic.png)

That means that a proper model of this DAC will be:

![Image06](images\DAC_Analysis\Image06.png)

As some current is needed to drive the LED, the current available to external resistances will be lower.

## DAC2 Open Circuit

We can first obtain the measurement data in open circuit.

![Image07](images\DAC_Analysis\Image07.png)

As the green **user LED** is connected to DAC2 output, you will se it turn on during the DC sweep.

In [ ]:
# MEASURE Open Circuit

# Repeat the DC sweep with the unloaded DAC2
dataOC_II = slab.dcSweep(2,0.1,3.3,0.1)

# Set DACs to zero
slab.zero()

# Obtain the output vector
VocII = dataOC_II[2];

# Response curve
slab.plot11(Vset,VocII,'Unloaded DAC2','Vset (V)','Vdac2 (V)')

We observe that we cannot reach $3.3 V$ due to the **LED** loading on DAC2.

So, even on unloaded condition, we cannot go over $3 V$.

We can now try to model this DAC using a modification of the previous model

In [ ]:
Rled = 510  # LED Resistance
Vled = 2    # We assume this ON Green LED voltage

# DAC2 unloaded model
# Gives the DAC2 voltage from Vset
def DAC2model(Vset):
    Vnii = Vset - Voffs
    if Vnii > Vled:
        Iout = (Vnii-Vled)/Rled
        Voa = Vled + Iout*(Rout+Rled)
        if Voa < Vdd:
            return Vnii 
        Iout = (Vdd-Vled)/(Rout+Rled)
        return Vled + Iout*Rled
    else:
        Voa = Vnii
        return Voa

# Apply the model to OC case
VocIIM = [DAC2model(V) for V in Vset]

# Plot VocII and VocIIM against Vset
slab.plot1n(Vset,[VocII,VocIIM],'Unloaded DAC2','Vset (V)','Vdac (V)',['Meas','Model'])

We can zoom in interactive mode to better see the graph details.

There is some error in the model, but it will probably mostly due to the LED voltage modelling.

## DAC2 Loaded

We can now repeat the measurements on DAC2 for the two load resistors.

![Image08](images\DAC_Analysis\Image08.png)

In [ ]:
# MEASURE with 330R

# Repeat the DC sweep with DAC2 loaded with 330R
dataR_II = slab.dcSweep(2,0.1,3.3,0.1)

# Set DACs to zero
slab.zero()

# Obtain the output vector
VrII = dataR_II[2];

# Response curve
slab.plot11(Vset,VrII,'DAC2 with 330R load','Vset (V)','Vdac2 (V)')

In [ ]:
# MEASURE with 470R

# Repeat the DC sweep with DAC2 loaded with 4700R
dataR2_II = slab.dcSweep(2,0.1,3.3,0.1)

# Set DACs to zero
slab.zero()

# Obtain the output vector
Vr2II = dataR2_II[2];

# Response curve
slab.plot11(Vset,Vr2II,'DAC2 with 470R load','Vset (V)','Vdac2 (V)')

## Curve Comparison

We can compare the DAC1 and DAC2 curves for the same load resistances

In [ ]:
# Compare DAC1 and DAC2 for 330R
slab.plot1n(Vset,[Vr,VrII],'DAC1 vs DAC2 for 330R','Vset (V)','Vdac (V)',['DAC1','DAC2'])

# Compare DAC1 and DAC2 for 470R
slab.plot1n(Vset,[Vr2,Vr2II],'DAC1 vs DAC2 for 470R','Vset (V)','Vdac (V)',['DAC1','DAC2'])

We can zoom in those curves to better see the details.

We can see that DAC2 has less drive capability, for both resistor values, due to the added load of the **user LED**. We could model DAC2 taking into account both the user LED and the external resistance but this is left as an exercise for the reader.

For now, you can just disconnect from the board

In [ ]:
# Disconnect from the board
slab.disconnect()

## Conclusion

In this **demo** project we have studied the DAC output curves and created a model for the DACs  
The model can be usefull to know when the DACs can be considered as ideal and when they need a proper non ideal modelling

## Document license

Copyright  ©  Vicente Jiménez (2018)  
This work is licensed under a Creative Common Attribution-ShareAlike 4.0 International license.  
This license is available at http://creativecommons.org/licenses/by-sa/4.0/

<img  src="images/cc_sa.png" width="200">